In [1]:
import tensorflow as tf
import numpy as np
import json
from mlff.data_prep import dataset_from_json, preprocessed_dataset_from_json
from mlff.models import NNEmbeddingModel
import os

In [ ]:
type_dict = {'Ni': 0, 'Pt': 1}

# Use smaller cutoff for preprocessed dataset!
#dataset_train = preprocessed_dataset_from_json('../data/train_data.json', type_dict, cutoff=5.006, batch_size=16)
#dataset_val = preprocessed_dataset_from_json('../data/val_data.json', type_dict, cutoff=5.006)
dataset_train = dataset_from_json('../data/train_data.json', type_dict, batch_size=20)
dataset_val = dataset_from_json('../data/val_data.json', type_dict)

In [ ]:
params = {
    ('A', 'PtPt'): 0.1602, ('A', 'NiPt'): 0.1346, ('A', 'NiNi'): 0.0845,
    ('xi', 'PtPt'): 2.1855, ('xi', 'NiPt'): 2.3338, ('xi', 'NiNi'): 1.405,
    ('p', 'PtPt'): 13.00, ('p', 'NiPt'): 14.838, ('p', 'NiNi'): 11.73,
    ('q', 'PtPt'): 3.13, ('q', 'NiPt'): 3.036, ('q', 'NiNi'): 1.93,
    ('r0', 'PtPt'): 2.77, ('r0', 'NiPt'): 2.63, ('r0', 'NiNi'): 2.49,
    ('cut_a', 'PtPt'): 4.08707719, ('cut_b', 'PtPt'): 5.0056268338740553,
    ('cut_a', 'NiPt'): 4.08707719, ('cut_b', 'NiPt'): 4.4340500673763259,
    ('cut_a', 'NiNi'): 3.62038672, ('cut_b', 'NiNi'): 4.4340500673763259,
    ('F_layers', 'Ni'): [15, 15], ('F_layers', 'Pt'): [15, 15]}
model = NNEmbeddingModel(['Ni', 'Pt'], params=params, reg=1e-5, build_forces=True)

In [4]:
class RaggedMeanSquaredError(tf.keras.losses.Loss):

    def call(self, y_true, y_pred):
        return tf.math.reduce_mean(tf.math.squared_difference(y_pred, y_true))

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=[RaggedMeanSquaredError(),
                    RaggedMeanSquaredError()],
             )

In [5]:
run_name = 'NNF/lambda1E-5/15-15/'
if not os.path.exists('./saved_models/%s' % run_name):
    os.makedirs('./saved_models/%s' % run_name)
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='./saved_models/%s/model.{epoch:03d}-{val_loss:.4f}.h5' % run_name, save_weights_only=True),
    tf.keras.callbacks.TensorBoard(log_dir='./logs/%s/' % run_name, profile_batch='10,20')
]

In [6]:
model.fit(x=dataset_train, validation_data=dataset_val, epochs=200, callbacks=my_callbacks, initial_epoch=0)

Epoch 1/200


/home/rmeyer/Development/py3.7/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


19/61 [========>.....................] - ETA: 1:16 - loss: 182.1325 - output_1_loss: 156.8344 - output_2_loss: 25.2978WARNING:tensorflow:From /home/rmeyer/Development/py3.7/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
61/61 [==============================] - 133s 2s/step - loss: 70.2329 - output_1_loss: 60.0869 - output_2_loss: 10.1457 - val_loss: 3.7217 - val_output_1_loss: 2.5363 - val_output_2_loss: 1.1850
Epoch 2/200
61/61 [==============================] - 127s 2s/step - loss: 1.9821 - output_1_loss: 1.4764 - output_2_loss: 0.5054 - val_loss: 0.7717 - val_output_1_loss: 0.5694 - val_output_2_loss: 0.2020
Epoch 3/200
61/61 [==============================] - 132s 2s/step - loss: 0.5138 - output_1_loss: 0.3760 - output_2_loss: 0.1375 - val_loss: 0.2671 - val_output_1_loss: 0.1545 - val_out

KeyboardInterrupt: 

In [8]:
model.weights[0:6:2]

[<tf.Variable 'A_NiNi:0' shape=(1,) dtype=float32, numpy=array([0.12552746], dtype=float32)>,
 <tf.Variable 'A_NiPt:0' shape=(1,) dtype=float32, numpy=array([0.200696], dtype=float32)>,
 <tf.Variable 'A_PtPt:0' shape=(1,) dtype=float32, numpy=array([0.22077666], dtype=float32)>]

In [9]:
model.weights[1:7:2]

[<tf.Variable 'p_NiNi:0' shape=(1,) dtype=float32, numpy=array([11.722518], dtype=float32)>,
 <tf.Variable 'p_NiPt:0' shape=(1,) dtype=float32, numpy=array([12.692806], dtype=float32)>,
 <tf.Variable 'p_PtPt:0' shape=(1,) dtype=float32, numpy=array([12.199522], dtype=float32)>]

In [22]:
model.weights[15:21:2]

[<tf.Variable 'xi_NiNi:0' shape=(1,) dtype=float32, numpy=array([1.7339661], dtype=float32)>,
 <tf.Variable 'xi_NiPt:0' shape=(1,) dtype=float32, numpy=array([2.1190224], dtype=float32)>,
 <tf.Variable 'xi_PtPt:0' shape=(1,) dtype=float32, numpy=array([2.2355506], dtype=float32)>]

In [21]:
model.weights[16:22:2]

[<tf.Variable 'q_NiNi:0' shape=(1,) dtype=float32, numpy=array([2.884065], dtype=float32)>,
 <tf.Variable 'q_NiPt:0' shape=(1,) dtype=float32, numpy=array([3.87875], dtype=float32)>,
 <tf.Variable 'q_PtPt:0' shape=(1,) dtype=float32, numpy=array([3.5221345], dtype=float32)>]